# Introduction
The code is split into parts, we save into csv file output at multiple places.
1. Part 1
   1. Used to extract features to fn-dataset2
   2. split across multiple files (18 files in mycase)
   3. the extract_range function can be used to extract all at once by giving full range
2. Load the fn-dataset2 and apply feature selection
   1. load csv files containing pattern extracted_features_*.csv from fn-dataset2
   2. Use pretrained DFP model to extract features and save it to  extracted_features_with_bmi.csv
3. Load extracted_features_with_bmi.csv and run linear reg after applying PCA
4. Classify based in BMI
5. Some tests with our pirctures (Does not seem to work for underwight well after applying PCA)
    1. Target Transformation (non-linear) testing


# PART 1

# Preprocessing & initial Feature Extraction

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from PIL import UnidentifiedImageError

import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd

import os

## Model

In [ ]:
# Step 1: Load Pre-trained InceptionV3 model (close variant to InceptionResNetV2)
model0 = models.inception_v3(pretrained=True)
model0.eval()  # Set model to evaluation mode

## preprocessing

In [ ]:
csv_path = '/kaggle/input/illinois-doc-labeled-faces-dataset/person.csv'
df = pd.read_csv(csv_path,sep = ';')

myD = df[['id','sex','height','weight']]
myD.head(8)

In [ ]:
# Convert height from inches to meters and weight from weight from pounds to kilograms
myD['height_m'] = myD['height'] * 0.0254
myD['weight_kg'] = myD['weight'] * 0.453592

 # Calculating BMI 
myD['BMI'] = myD['weight_kg'] / (myD['height_m'] ** 2)

# Encode gender
myD['sex'] = myD['sex'].fillna('')
myD['Gender'] = myD['sex'].apply(lambda x: 1 if str(x).strip().lower() == 'male' else 0)

## add path to front and side images

In [ ]:
import os
# Define image directories
front_dir = '/kaggle/input/illinois-doc-labeled-faces-dataset/front/front'
side_dir = '/kaggle/input/illinois-doc-labeled-faces-dataset/side/side'

# Add image paths to the DataFrame
myD['front_image'] = myD['id'].apply(lambda x: os.path.join(front_dir, f'{x}.jpg'))
myD['side_image'] = myD['id'].apply(lambda x: os.path.join(side_dir, f'{x}.jpg'))

In [ ]:
myD.head(5)

In [ ]:
# Define image preprocessing pipeline for PyTorch
preprocess = transforms.Compose([
    transforms.Resize(299),  # Resize to 299x299 for InceptionV3
    transforms.CenterCrop(299),  # Crop to 299x299
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

# Assume 'model' is your pretrained InceptionV3 or any model you're using.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model0.to(device)  # Move the model to the GPU or CPU


# Function to load and preprocess images
def load_image(path):
    if os.path.exists(path) and path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        try:
            img = Image.open(path)
            img = preprocess(img)  # Apply preprocessing
            img = img.unsqueeze(0)  # Add batch dimension
            return img
        except UnidentifiedImageError:
            print(f"Unidentified image error with file: {path}")
            return None
        except Exception as e:
            print(f"Error loading image {path}: {e}")
            return None
    else:
        print(f"Skipping non-image file: {path}")
        return None

# Function to extract features using InceptionV3 (FaceNet-like)
def extract_features(image_path):
    img = load_image(image_path)
    if img is not None:
        img = img.to(device)  # Move the image to the same device as the model
        with torch.no_grad():  # No need to track gradients during inference
            features = model0(img)  # Extract features (raw output)
        features = features.squeeze().cpu().numpy()  # Move back to CPU and convert to numpy
        return features
    return None

# Function to process a range of entries with automatic file naming and feature metrics
def extract_range(start_index, end_index, data):
    # Initialize lists to store features and BMI values
    front_features = []
    side_features = []
    bmi_values = []

    # Initialize variables to store feature statistics
    feature_lengths = []
    feature_means = []
    feature_stds = []

    # Loop over the specified range of rows in the data DataFrame
    for idx, row in data.iloc[start_index:end_index].iterrows():
        front_img_features = extract_features(row['front_image'])
        side_img_features = extract_features(row['side_image'])
        
        if front_img_features is not None and side_img_features is not None:
            # Combine features from both views into a single feature vector
            combined_features = np.concatenate((front_img_features, side_img_features))
            front_features.append(combined_features)
            bmi_values.append(row['BMI'])
            
            # Collect feature statistics
            feature_lengths.append(len(combined_features))
            feature_means.append(np.mean(combined_features))
            feature_stds.append(np.std(combined_features))

    # Convert to numpy arrays
    X = np.array(front_features)
    y = np.array(bmi_values)
    
    # Create a DataFrame from the extracted features and BMI values
    df = pd.DataFrame(X)
    df['BMI'] = y
    
    # Calculate the feature statistics
    feature_stats = {
        'num_features': X.shape[1],  # Total number of features per row
        'mean_features': np.mean(X),  # Mean of all features
        'std_features': np.std(X),    # Standard deviation of all features
        'mean_per_feature': np.mean(feature_means),  # Mean of individual feature means
        'std_per_feature': np.mean(feature_stds),    # Mean of individual feature stds
        'min_feature_length': np.min(feature_lengths),  # Min feature length
        'max_feature_length': np.max(feature_lengths)   # Max feature length
    }
    
    print("Feature Statistics:", feature_stats)

    # Generate a filename based on the range of rows processed
    output_filename = f"extracted_features_{start_index}_{end_index}.csv"
    
    # Save the DataFrame to CSV
    df.to_csv(output_filename, index=False)
    
    print(f"Saved extracted features to {output_filename}")
    return feature_stats  # Return statistics for further analysis if needed

## extract in smaller batches

In [ ]:
# Call extract_range with range
data = myD.copy()
# feature_stats = extract_range(40001, 50000, data)


###############  ADD RANDOM here

###############

# PART 2

# Load FaceNet Features

In [ ]:
import pandas as pd
import glob

# Define the path to your CSV files
path = '/kaggle/input/fn-dataset2/extracted_features_*.csv'

# Use glob to match the pattern
csv_files = glob.glob(path)

print(csv_files)
# Read each CSV file and load into a DataFrame
dfs = [pd.read_csv(file) for file in csv_files]

# If you want to combine them into one DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Display the combined DataFrame
print(combined_df.shape)


# Apply CNN feature subset extractor

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Remove rows with NaN values in the 'BMI' column
combined_df_cleaned = combined_df.dropna(subset=['BMI'])

# Now, drop the 'BMI' column to create X
X = combined_df_cleaned.drop(columns=['BMI'])

# Extract the target variable 'BMI' into y
y = combined_df_cleaned['BMI']

Y = y.copy()
# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Reshape X for CNN (adding a channel dimension)
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)
print(X_reshaped.shape)

if np.any(np.isnan(y)):
    print("Warning: y contains NaN values.")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np

# Load the saved model
feature_extractor = load_model('/kaggle/input/dfpv2.1/keras/prml_bmi_33/1/DFPv2.h5')

# Extract features
X_features = feature_extractor.predict(X_reshaped)

# Flatten the features if they're multidimensional
X_features_flat = X_features.reshape(X_features.shape[0], -1)

# Create a DataFrame with the extracted features and BMI
feature_cols = [f'feature_{i}' for i in range(X_features_flat.shape[1])]
features_df = pd.DataFrame(X_features_flat, columns=feature_cols)

# Ensure Y is converted to the correct type and shape
Y_cleaned = np.array(Y).flatten().astype(float)

# Add BMI column ensuring no type conversion issues
features_df['BMI'] = Y_cleaned


# Save to CSV
features_df.to_csv('/kaggle/working/extracted_features_with_bmi.csv', index=False)
print("Features shape:", X_features_flat.shape)
print("Features saved to 'extracted_features_with_bmi.csv'")

## load features for linear regression; split 80-20

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

# Step 1: Load the dataset
data = pd.read_csv('/kaggle/working/extracted_features_with_bmi.csv')

# Step 2: Prepare features (X) and target (y)
# Assuming the last column is the target variable (e.g., 'BMI')
X = data.drop(columns=['BMI'])  # Drop target column
y = data['BMI']  # Assuming 'BMI' is the column you want to predict

# Step 3: Apply PCA
pca = PCA(n_components=0.90)  
X_pca = pca.fit_transform(X)
# X_pca = X

# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=38)

# Step 5: Perform Linear Regression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Step 6: Make predictions
y_train_pred = regressor.predict(X_train)
y_test_pred = regressor.predict(X_test)

# Step 7: Calculate Mean Squared Error (MSE) for both train and test sets
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

# Step 8: Calculate Mean Absolute Error (MAE) for both train and test sets
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

# Step 9: Calculate R-squared (R²) for both train and test sets
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Step 10: Calculate Pearson correlation coefficient for both train and test sets
train_pearson, _ = pearsonr(y_train, y_train_pred)
test_pearson, _ = pearsonr(y_test, y_test_pred)

# Create a dictionary with the metrics for training and testing
metrics = {
    "Metric": ["MSE", "MAE", "R²", "Pearson Correlation"],
    "Training": [train_mse, train_mae, train_r2, train_pearson],
    "Testing": [test_mse, test_mae, test_r2, test_pearson]
}

# Create a DataFrame
metrics_df = pd.DataFrame(metrics)

# Print the DataFrame as a table
print(metrics_df)

# Step 11: Plot results
# Plot for training data
plt.figure(figsize=(14, 6))

# Plot Training Data: Actual vs Predicted
plt.subplot(1, 2, 1)
plt.scatter(y_train, y_train_pred, color='blue', label='Training Data')
plt.plot([min(y_train), max(y_train)], [min(y_train), max(y_train)], color='red', lw=2, label='Ideal Fit')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Training Data: Actual vs Predicted')
plt.legend()

# Plot Testing Data: Actual vs Predicted
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_test_pred, color='green', label='Testing Data')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', lw=2, label='Ideal Fit')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Testing Data: Actual vs Predicted')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
def classify_bmi(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi < 25:
        return "Normal"
    else:
        return "Overweight (Obese)"

# Classify actual and predicted BMI values
y_train_class = [classify_bmi(bmi) for bmi in y_train]
y_train_pred_class = [classify_bmi(bmi) for bmi in y_train_pred]
y_test_class = [classify_bmi(bmi) for bmi in y_test]
y_test_pred_class = [classify_bmi(bmi) for bmi in y_test_pred]



# Step 13: Calculate accuracy for BMI classification
train_accuracy = accuracy_score(y_train_class, y_train_pred_class)
test_accuracy = accuracy_score(y_test_class, y_test_pred_class)

print(f"Training BMI Classification Accuracy: {train_accuracy * 100:.2f}%")
print(f"Testing BMI Classification Accuracy: {test_accuracy * 100:.2f}%")

# Step 14: Plot confusion matrices for classification accuracy

# Confusion Matrix for Training Data
train_cm = confusion_matrix(y_train_class, y_train_pred_class, labels=["Underweight", "Normal", "Overweight (Obese)"])
# Confusion Matrix for Testing Data
test_cm = confusion_matrix(y_test_class, y_test_pred_class, labels=["Underweight", "Normal", "Overweight (Obese)"])

# Plot for Training Data Confusion Matrix
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
sns.heatmap(train_cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Underweight", "Normal", "Overweight (Obese)"], yticklabels=["Underweight", "Normal", "Overweight (Obese)"])
plt.title("Training Data Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")

# Plot for Testing Data Confusion Matrix
plt.subplot(1, 2, 2)
sns.heatmap(test_cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Underweight", "Normal", "Overweight (Obese)"], yticklabels=["Underweight", "Normal", "Overweight (Obese)"])
plt.title("Testing Data Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")

plt.tight_layout()
plt.show()

# Testing

## predict BMI

# OURS

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import torch
from sklearn.preprocessing import StandardScaler
import pandas as pd

def extract_features2(image_path, model0, device):
    """
    This function takes an image path, model0 (InceptionV3 model), and device (CPU/GPU), 
    loads and preprocesses the image, and returns the extracted features.
    """
    img = load_image(image_path)
    if img is not None:
        img = img.to(device)  # Move the image to the same device as the model
        with torch.no_grad():  # No need to track gradients during inference
            features = model0(img)  # Extract features (raw output)
        features = features.squeeze().cpu().numpy()  # Move back to CPU and convert to numpy
        return features
    return None

def oursTst(front_img_path, side_img_path, bmi_value, regressor, feature_extractor, scaler, model0, device):
    """
    This function takes in paths for front and side images, BMI value, a regressor model, 
    a feature extractor model, and a scaler, and returns predictions and actual BMI values.
    """
    # Load and preprocess the front and side images
    front_img_features = extract_features2(front_img_path, model0, device)
    side_img_features = extract_features2(side_img_path, model0, device)

    if front_img_features is not None and side_img_features is not None:
        # Combine features from both views into a single feature vector
        combined_features = np.concatenate((front_img_features, side_img_features))

        # Normalize the features
        rand_X_scaled = scaler.transform([combined_features])
        
        
        # Reshape X for CNN (adding a channel dimension)
        rand_X_reshaped = rand_X_scaled.reshape(rand_X_scaled.shape[0], rand_X_scaled.shape[1], 1)

            
        # Extract features from the CNN model
        extracted_features = feature_extractor.predict(rand_X_reshaped)

        extracted_features = pca.transform(extracted_features)
        # Make predictions
        pred = regressor.predict(extracted_features)[0]
        act = int(bmi_value)  # The actual BMI value

        # Classify both predicted and actual BMI
        pred_class = classify_bmi(pred)
        act_class = classify_bmi(act)

        # Plot the front and side images with captions
        img_front = Image.open(front_img_path)
        img_side = Image.open(side_img_path)

        # Create a figure with two subplots (side by side)
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns

        # Plot the front image
        axes[0].imshow(img_front)
        axes[0].axis('off')  # Hide axes
        axes[0].set_title('Front Image')

        # Plot the side image
        axes[1].imshow(img_side)
        axes[1].axis('off')  # Hide axes
        axes[1].set_title('Side Image')

        # Add caption below each image
        # axes[0].text(0.5, -0.1, f'Predicted: {pred}, Actual: {act}', ha='center', va='center', transform=axes[0].transAxes, fontsize=12)
        # axes[1].text(0.5, -0.1, f'Predicted: {pred}, Actual: {act}', ha='center', va='center', transform=axes[1].transAxes, fontsize=12)

        axes[0].text(0.5, -0.1, f'Predicted: {pred} ({pred_class}),\n Actual: {act} ({act_class})', 
                     ha='center', va='center', transform=axes[0].transAxes, fontsize=12)
        axes[1].text(0.5, -0.1, f'Predicted: {pred} ({pred_class}),\n Actual: {act} ({act_class})', 
                     ha='center', va='center', transform=axes[1].transAxes, fontsize=12)

        # Show the plot
        plt.tight_layout()
        plt.show()

        return pred, act
    else:
        print("Error: Could not extract features from one or both images.")
        return None, None


## ASHIQ

In [ ]:
pred, act = oursTst("/kaggle/input/penguinsset/ashiq_front.jpeg", "/kaggle/input/penguinsset/ashiq_side.jpeg", 18, regressor, feature_extractor, scaler, model0, device)


## AJMAL

In [ ]:
pred, act = oursTst("/kaggle/input/penguinsset/ajmal_front.jpeg", "/kaggle/input/penguinsset/ajmal_side.jpeg", 20, regressor, feature_extractor, scaler, model0, device)


## Rand from Dataset

In [ ]:
def wrapper(x):
    imgPath = myD.iloc[x].id
    # Specify the image paths
    imgPath_front = f'/kaggle/input/illinois-doc-labeled-faces-dataset/front/front/{imgPath}.jpg'
    imgPath_side = f'/kaggle/input/illinois-doc-labeled-faces-dataset/side/side/{imgPath}.jpg'
    pred, act = oursTst(imgPath_front,imgPath_side, myD.iloc[x].BMI , regressor, feature_extractor, scaler, model0, device)


In [ ]:
wrapper(600)


In [ ]:
wrapper(700)


In [ ]:
wrapper(8000)

# Target Transformation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

# Load the dataset
data = pd.read_csv('/kaggle/working/extracted_features_with_bmi.csv')


# Conditional adjustment function for target
def adjust_target(target):
    return np.where(target < 20, target + 50/target, target + 0)

def invert_adjustment(target):
    return np.where(target < 25, target - 50/target, 
                    np.where((target >= 25) & (target <= 30), target-30/target, target))



# Prepare features (X) and target (y)
X = data.drop(columns=['BMI'])  # Drop target column
y = adjust_target(data['BMI'])  # Apply conditional adjustment

# Apply PCA
pca = PCA(n_components=0.90)  
X_pca = pca.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=38)

# Perform Linear Regression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Make predictions and invert adjustment
y_train_pred = invert_adjustment(regressor.predict(X_train))
y_test_pred = invert_adjustment(regressor.predict(X_test))

# Original target values (for comparison)
y_train_original = invert_adjustment(y_train)
y_test_original = invert_adjustment(y_test)

# Calculate metrics
train_mse = mean_squared_error(y_train_original, y_train_pred)
test_mse = mean_squared_error(y_test_original, y_test_pred)

train_mae = mean_absolute_error(y_train_original, y_train_pred)
test_mae = mean_absolute_error(y_test_original, y_test_pred)

train_r2 = r2_score(y_train_original, y_train_pred)
test_r2 = r2_score(y_test_original, y_test_pred)

train_pearson, _ = pearsonr(y_train_original, y_train_pred)
test_pearson, _ = pearsonr(y_test_original, y_test_pred)

# Create metrics DataFrame
metrics = {
    "Metric": ["MSE", "MAE", "R²", "Pearson Correlation"],
    "Training": [train_mse, train_mae, train_r2, train_pearson],
    "Testing": [test_mse, test_mae, test_r2, test_pearson]
}
metrics_df = pd.DataFrame(metrics)

# Print the metrics
print(metrics_df)

# Plotting
plt.figure(figsize=(14, 6))

# Plot Training Data: Actual vs Predicted
plt.subplot(1, 2, 1)
plt.scatter(y_train_original, y_train_pred, color='blue', label='Training Data')
plt.plot([min(y_train_original), max(y_train_original)], 
         [min(y_train_original), max(y_train_original)], 
         color='red', lw=2, label='Ideal Fit')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Training Data: Actual vs Predicted')
plt.legend()

# Plot Testing Data: Actual vs Predicted
plt.subplot(1, 2, 2)
plt.scatter(y_test_original, y_test_pred, color='green', label='Testing Data')
plt.plot([min(y_test_original), max(y_test_original)], 
         [min(y_test_original), max(y_test_original)], 
         color='red', lw=2, label='Ideal Fit')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Testing Data: Actual vs Predicted')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
def oursTst(front_img_path, side_img_path, bmi_value, regressor, feature_extractor, scaler, model0, device):

    # Load and preprocess the front and side images
    front_img_features = extract_features2(front_img_path, model0, device)
    side_img_features = extract_features2(side_img_path, model0, device)
    
    if front_img_features is not None and side_img_features is not None:
        # Combine features from both views into a single feature vector
        combined_features = np.concatenate((front_img_features, side_img_features))
        
        # Normalize the features
        rand_X_scaled = scaler.transform([combined_features])
        
        # Reshape X for CNN (adding a channel dimension)
        rand_X_reshaped = rand_X_scaled.reshape(rand_X_scaled.shape[0], rand_X_scaled.shape[1], 1)
        
        # Extract features from the CNN model
        extracted_features = feature_extractor.predict(rand_X_reshaped)
        extracted_features = pca.transform(extracted_features)
        
        # Adjust the target before prediction
        adjusted_bmi = adjust_target(bmi_value)
        
        # Make predictions and invert the adjustment
        pred_adjusted = regressor.predict(extracted_features)[0]
        pred = invert_adjustment(pred_adjusted)
        
        act = int(bmi_value)  # The actual BMI value
        
        # Classify both predicted and actual BMI
        pred_class = classify_bmi(pred)
        act_class = classify_bmi(act)
        
        # Plot the front and side images with captions
        img_front = Image.open(front_img_path)
        img_side = Image.open(side_img_path)
        
        # Create a figure with two subplots (side by side)
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns
        
        # Plot the front image
        axes[0].imshow(img_front)
        axes[0].axis('off')  # Hide axes
        axes[0].set_title('Front Image')
        
        # Plot the side image
        axes[1].imshow(img_side)
        axes[1].axis('off')  # Hide axes
        axes[1].set_title('Side Image')
        
        # Add caption below each image
        caption = f'Predicted: {pred:.2f} ({pred_class}),\n Actual: {act} ({act_class})'
        axes[0].text(0.5, -0.1, caption, 
                     ha='center', va='center', transform=axes[0].transAxes, fontsize=12)
        axes[1].text(0.5, -0.1, caption, 
                     ha='center', va='center', transform=axes[1].transAxes, fontsize=12)
        
        # Show the plot
        plt.tight_layout()
        plt.show()
        
        return pred, act
    else:
        print("Error: Could not extract features from one or both images.")
        return None, None
pred, act = oursTst("/kaggle/input/penguinsset/ashiq_front.jpeg", "/kaggle/input/penguinsset/ashiq_side.jpeg", 18, regressor, feature_extractor, scaler, model0, device)
pred, act = oursTst("/kaggle/input/penguinsset/ajmal_front.jpeg", "/kaggle/input/penguinsset/ajmal_side.jpeg", 20, regressor, feature_extractor, scaler, model0, device)

def wrapper(x):
    imgPath = myD.iloc[x].id
    # Specify the image paths
    imgPath_front = f'/kaggle/input/illinois-doc-labeled-faces-dataset/front/front/{imgPath}.jpg'
    imgPath_side = f'/kaggle/input/illinois-doc-labeled-faces-dataset/side/side/{imgPath}.jpg'
    pred, act = oursTst(imgPath_front,imgPath_side, myD.iloc[x].BMI , regressor, feature_extractor, scaler, model0, device)


wrapper(150)
wrapper(300)

In [ ]:
wrapper(12000)